<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/session.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Sessionクラス解説

Session クラスはAgentの各イベント(on_tick, on_clock, on_update)時に引数として送られてきます。

内部に現在時刻 `timestamp`を保持することによりバックテスト、フォーワードテスト、本番どれであっても同一処理のAgent(BOT)が動作するようになっています。

このSessionのインスタンスを利用してAgent内の取引ロジックを開発してください。
使えるプロパティ／メソッドについて説明します。

ユーザがSessionオブジェクトを直接自分で生成することはありません。

## プロパティ

### 実行状況表すプロパティ

* `timestamp` 現在時刻をUnixtime(us)で保存取得します。
* `execute_mode` 現在の実行モードを返す。`Real`(本番), `Dummy`(バックテスト), `Dry`(フォーワードテスト）のどれかが得られる。

### 市況を示すプロパティ
* `board` Marketの板情報(`bit`, `ask`)でまとめて取得する
* `last_price` 直近の約定履歴から想定される`bit`, `ask`の最良値（`bit`, `ask`)でまとめて取得する。板情報が取れないときに代わりに利用する。

### 自分のオーダー表すプロパティ
* `buy_orders` 板にある未約定買いオーダー（一覧）
* `buy_order_amount` 板にある未約定買いオーダー注文量合計
* `sell_orders` 板にある未約定売りオーダー（一覧）
* `sell_order_amount` 板にある未約定売りオーダー注文量合計

### 仮想ポジション・利益を表すプロパティ
セッションがはじまってからのポジション・利益を取得する。
* `position` 仮想ポジション

### ログクラスを取得するプロパティ
実行終了後、ログを分析するためにセッションからログを取り出すプロパティ

* `log` 実行結果が入ったLoggerクラスのオブジェクトが取得できる。利用方法は[ログクラスマニュアル 参照](./logger.ipynb)

## メソッド

### 市況を得るメソッド

#### ローソク足
使い方
```
session.ohlcv(
    interval=60,    # 足の幅を秒数で指定
    count=5         # 足の本数を指定
)
```

戻り値：　OHLCVの値がPolarsのDataFrame形式で得られる。

---

### オーダーメソッド

#### 成り行き注文(market_order)
使い方
```
session.market_order(
    side='Buy',     # 売り`Buy`, 買い`Sell`を指定
    size=0.01       # オーダーサイズを指定
)
```
戻り値：作成されたオーダーオブジェクト（板の状況にあわせ分割された結果が配列になってきます）


#### 指値注文(limit_order)
使い方
```
session.limit_order(
    side='Buy',     # 売り`Buy`, 買い`Sell`を指定    
    price=40000,    # 指値を指定
    size=0.01       # オーダーサイズを指定
)
```
戻り値：作成されたオーダーオブジェクト（１行のみですが、他のオーダメソッドにあわせ配列形式）

#### キャンセルオーダー(cancel_order)
使い方
```
session.cancel_order(
    order_id='order_id_0001'    #オーダIDを文字列で指定。
)
```

戻り値：キャンセルされたオーダオブジェクト（１行のみですが、他のオーダメソッドにあわせ配列形式）

#### 期限切れ(expire_order)
使い方
```
session.expire_order(
    ttl_sec=60*10       #期限切れにする期間を秒で指定（この例では10分以上前のオーダをキャンセル）
)
```

戻り値(bool)
* `true` 期限切れがオーダーあった場合
* `false`　期限切れオーダーがなかった場合


## 以下テストを兼ねたSessionオブジェクトの内部動作確認

通常ユーザはSessionクラスを生成しません。Agentにおくられてきたオブジェクトを利用するだけです。

In [ ]:
import rbot

# バージョン確認

print("version =", rbot.__version__)

In [ ]:
from rbot import Session
from rbot import Runner
from rbot import BinanceMarket
from rbot import BinanceConfig
from rbot import ExecuteMode
from rbot import NOW, HHMM, SEC, time_string


#from rbot import MarketMessage

#from rbot import Trade
#from rbot import OrderSide

from rbot import init_log

In [ ]:
from IPython.display import HTML
from json2html import json2html
import json

In [ ]:

config = BinanceConfig.BTCUSDT
market = BinanceMarket(config)




In [ ]:
class DummyAgent:
    def on_init(self, session):
        session.clock_interval_sec = 10
        
    def on_clock(self, session, clock):
        print("現在時刻", session.timestamp, time_string(session.timestamp))
        print("実行モード", session.execute_mode)
        print("")

    
        if session.execute_mode == "Dry":
            print("Dryモードではいた情報が取れるので表示します")
            bit, ask = session.board
            print("板情報", bit, ask)
            return        
            
        bit_edge, ask_edge = session.last_price
        print("最終約定価格", bit_edge, ask_edge)
        
        raise Exception("stop agent")

In [ ]:
runner = Runner()

agent = DummyAgent()

runner.back_test(agent=agent, market=market)

In [ ]:
class SimpleOrderAgent:
    def __init__(self):
        self.index = 0
        
    def on_init(self, session):
        session.clock_interval_sec = 60*10      # 10分ごとに呼び出される
        print("実行モード", session.execute_mode)
        
    def on_clock(self, session, clock):
        print("---------------------------")
        print("現在時刻", session.timestamp, time_string(session.timestamp))
        print("売り注文", session.sell_orders)
        print("買い注文", session.buy_orders)
        print("ポジション", session.position)

        self.index += 1
        
        if self.index == 1:
            print("成り行き注文テスト(売り)")
            buy_order = session.market_order("Buy", 0.001)
            print("buy_order", buy_order)

        elif self.index == 2:
            print("成り行き注文テスト(買い)")
            sell_order= session.market_order("Sell", 0.001)
            print("sell_order", sell_order)
            
            print("指値注文テスト")            
            pass    
        elif self.index == 3:
            print("指値注文テスト")
            bit, ask = session.last_price
            buy_order = session.limit_order("Buy", bit-100, 0.001)
            print("buy_order", buy_order)
            
            sell_order = session.limit_order("Sell", ask+100, 0.001)
            print("sell_order", sell_order)

        elif self.index == 4:
            print("Cancelテスト(SELL)")
            if session.sell_orders:
                order = session.sell_orders[0]
                cancel_order = session.cancel_order(order.order_id)
                print("cancel_order=", cancel_order)

        elif self.index == 5:
            print("Exipreテスト1")
            
            if session.expire_order(60*30): # 30分前の注文をキャンセル
                print("expire_order exist")
            else:
                print("expire_order not exist")

        elif self.index == 6:
            print("Exipreテスト2")
            
            if session.expire_order(60*10): # 10分前の注文をキャンセル
                print("expire_order exist")
            else:
                print("expire_order not exist")
        else:
            raise Exception("stop agent")
        
        print("---------------------------")        
    
    def on_update(self, session, update):
        print("更新情報", update)

In [ ]:
runner = Runner()

agent = SimpleOrderAgent()

runner.back_test(agent=agent, market=market)

In [ ]:
from rbot import MarketMessage
from rbot import Trade
from rbot import OrderSide
from rbot import LogStatus

target = NOW() - HHMM(6,0)
order = market.select_trades(target-SEC(10), target)

t = int(time.mktime(order['timestamp'][0].timetuple()))*1000
price =order['price'][0]
size = order['size'][0]

session.on_message(MarketMessage.from_trade(Trade(t, OrderSide.Sell, price, size, LogStatus.UnFix, "OrderId-1")))



## さらに細かい内部テスト



In [ ]:
HTML(json2html.convert(session.__str__()))

In [ ]:
# 内部処理、バックテスト用にログを読み取らせると市況がUpdateされる。
from rbot import NOW
import rbot

trade = MarketMessage.from_trade(Trade(NOW(), OrderSide.Buy, 100, 1, "OrderId-1"))
session.on_message(trade)

HTML(json2html.convert(session.__str__()))
    

In [ ]:
session.account

In [ ]:
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99, 1, "OrderId-2")))

HTML(json2html.convert(session.__str__()))

In [ ]:
# Agent用メソッド：　オーダーを発行すると,処理中オーダーにキュー(dummy_q)される
session.limit_order("Buy", 
                    100,    #価格
                    50,     # サイズ
)

HTML(json2html.convert(session.__str__()))

In [ ]:
session.dummy_q

In [ ]:
session.log_df

In [ ]:
# あたらしいメッセージがやってくるとQにあったオーダーが処理される。
# 買オーダー(100)よりも高い売り(101.0)がおこなわれてもオーダーには影響がない。
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 101, 50, "OrderId-2")))
HTML(json2html.convert(session.__str__()))

In [ ]:
session.dummy_q

In [ ]:
session.log_df

In [ ]:
# 買オーダー（１００）と同じ価格の売りがおこなわれているときも影響がない
# 本来は時間が経って板の最後までまわってきたら約定するが、辛めに判定
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 100.0, 50, "OrderId-3")))
HTML(json2html.convert(session.__str__()))

In [ ]:
# 買オーダー(100)よりも低い価格の売りがでてきたら、部分約定する（今回の例では、サイズ１が約定して49が残っている）
# ただし約定ログは分割せずremain_sizeをへらして全部約定のときにログを出す（あとで、まとめて約定したことにする）
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99.9, 1, "OrderId-4")))
HTML(json2html.convert(session.__str__()))

# TODO: アカウント情報のUpdate

In [ ]:
session.log_df

In [ ]:
# のこり49の売りが発生すると全部約定する（注残リストからはなくなる）
session.on_message(MarketMessage.from_trade(Trade(NOW(), OrderSide.Sell, 99.9, 49, "OrderId-5")))
HTML(json2html.convert(session.__str__()))

In [ ]:
session.log_df

# まとめ

以上のような動きを約定ログ全部なめてバックテストを行なっています。

（ここまで細かくやる必要が果たしてあったのかは、これからのBot開発でわかるようになるはず）